In [85]:
#Set up stuff from notebook
import os
import torch
import torch.nn as nn
import einops
from fancy_einsum import einsum
import tqdm.auto as tqdm
import plotly.express as px
from jaxtyping import Float
from functools import partial
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, FactoredMatrix

DEVELOPMENT_MODE = False
# Detect if we're running in Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
except:
    IN_COLAB = False

# Install if in Colab
if IN_COLAB:
    %pip install transformer_lens
    %pip install circuitsvis
    # Install a faster Node version
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs  # noqa

# Hot reload in development mode & not running on the CD
if not IN_COLAB:
    from IPython import get_ipython
    ip = get_ipython()
    if not ip.extension_manager.loaded:
        ip.extension_manager.load('autoreload')
        %autoreload 2

IN_GITHUB = os.getenv("GITHUB_ACTIONS") == "true"
print("works")

# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh

import plotly.io as pio

pio.renderers.default = "notebook_connected"
print(f"Using renderer: {pio.renderers.default}")

import circuitsvis as cv
# Testing that the library works
cv.examples.hello("Neel")

torch.set_grad_enabled(False)

def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

device = utils.get_device()

works
Using renderer: notebook_connected


In [86]:
def get_params(model, show_param = False):
    count = 0
    parameters = []
    names = []
    for name, param in model.named_parameters():
        if show_param == True:
            print(name, param.size())
        names.append([name, param.size()])
        parameters.append(param.data)
        count += 1
    print(f"Model has {count} named parameters")
    return parameters, names


def unpack_names(names, first = 0, last = None):
    if last == None:
        for item in names[first:len(names)-1]:
            print(f"Param: {item[0]:<40} Dimensions of tensor: {item[1]:<30}")
    else:
        for item in names[first: last]:
            print(f"Param: {item[0]:<40} Dimensions of tensor: {str(item[1]):<30}")


def compare(model_1, model_2):
    different = []
    model_1_params, model_1_names = get_params(model_1)
    model_2_params, model_2_names = get_params(model_2)

    if len(model_1_params) != len(model_2_params):
        print("There are a different number of parameters between the models. Please check model config")
        print(f"Model 1 has {len(model_1_params)} parameters. Model 2 has {len(model_2_params)}")

    for i in range(min(len(model_1_params),(len(model_2_params)))):
        if model_1_names[i][1] != model_2_names[i][1]:
            different.append([model_1_names[i], model_2_names[i], ("index", i)])

    print(f"There are {len(different)} parameters that are different from each other")
    return different 

In [87]:
from transformers import GPTJForCausalLM        
model_path = r"C:\Users\allan\ResearchStuff\checkpoint-1953"

gptj = GPTJForCausalLM.from_pretrained(model_path)

In [88]:
print(gptj)
gptj.config

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): Embedding(50257, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPTJBlock(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): Linear(in_features=768, out_features=768, bias=False)
          (v_proj): Linear(in_features=768, out_features=768, bias=False)
          (q_proj): Linear(in_features=768, out_features=768, bias=False)
          (out_proj): Linear(in_features=768, out_features=768, bias=False)
        )
        (mlp): GPTJMLP(
          (fc_in): Linear(in_features=768, out_features=3072, bias=True)
          (fc_out): Linear(in_features=3072, out_features=768, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((7

GPTJConfig {
  "_name_or_path": "C:\\Users\\allan\\ResearchStuff\\checkpoint-1953",
  "activation_function": "gelu_new",
  "architectures": [
    "GPTJForCausalLM"
  ],
  "attn_pdrop": 0.0,
  "bos_token_id": 50256,
  "embd_pdrop": 0.0,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gptj",
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "resid_pdrop": 0.0,
  "rotary_dim": 64,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.45.2",
  "use_cache": true,
  "vocab_size": 50257
}

In [89]:
gptj_params, gptj_names = get_params(gptj)

Model has 125 named parameters


In [90]:
unpack_names(gptj_names, 0, 30)

Param: transformer.wte.weight                   Dimensions of tensor: torch.Size([50257, 768])      
Param: transformer.h.0.ln_1.weight              Dimensions of tensor: torch.Size([768])             
Param: transformer.h.0.ln_1.bias                Dimensions of tensor: torch.Size([768])             
Param: transformer.h.0.attn.k_proj.weight       Dimensions of tensor: torch.Size([768, 768])        
Param: transformer.h.0.attn.v_proj.weight       Dimensions of tensor: torch.Size([768, 768])        
Param: transformer.h.0.attn.q_proj.weight       Dimensions of tensor: torch.Size([768, 768])        
Param: transformer.h.0.attn.out_proj.weight     Dimensions of tensor: torch.Size([768, 768])        
Param: transformer.h.0.mlp.fc_in.weight         Dimensions of tensor: torch.Size([3072, 768])       
Param: transformer.h.0.mlp.fc_in.bias           Dimensions of tensor: torch.Size([3072])            
Param: transformer.h.0.mlp.fc_out.weight        Dimensions of tensor: torch.Size([768, 3072

In [ ]:
from transformer_lens import HookedTransformerConfig

config = {
    "d_model" : 768,
    "d_head" : 64,
    "n_layers" : 12,
    "n_ctx" : 1024,
    "n_heads" : 12,
    "d_mlp" : 3072, 
    "d_vocab" : 50257,
    "act_fn" : "gelu_new", 
    "eps" : 1e-05,
    "normalization_type" : "LN", 
    "positional_embedding_type" : "rotary",
    "rotary_dim": 64,
    "post_embedding_ln": False

}


hooked = HookedTransformer(config)

In [97]:
hooked_params, hooked_names = get_params(hooked)

Model has 197 named parameters


In [100]:
unpack_names(hooked_names, first = 0, last = 30)

Param: embed.W_E                                Dimensions of tensor: torch.Size([50257, 768])      
Param: blocks.0.ln1.w                           Dimensions of tensor: torch.Size([768])             
Param: blocks.0.ln1.b                           Dimensions of tensor: torch.Size([768])             
Param: blocks.0.ln2.w                           Dimensions of tensor: torch.Size([768])             
Param: blocks.0.ln2.b                           Dimensions of tensor: torch.Size([768])             
Param: blocks.0.attn.W_Q                        Dimensions of tensor: torch.Size([12, 768, 64])     
Param: blocks.0.attn.W_O                        Dimensions of tensor: torch.Size([12, 64, 768])     
Param: blocks.0.attn.b_Q                        Dimensions of tensor: torch.Size([12, 64])          
Param: blocks.0.attn.b_O                        Dimensions of tensor: torch.Size([768])             
Param: blocks.0.attn.W_K                        Dimensions of tensor: torch.Size([12, 768, 

In [102]:
compare(hooked, gptj)

Model has 197 named parameters
Model has 125 named parameters
There are a different number of parameters between the models. Please check model config
Model 1 has 197 parameters. Model 2 has 125
There are 105 parameters that are different from each other


[[['blocks.0.ln2.w', torch.Size([768])],
  ['transformer.h.0.attn.k_proj.weight', torch.Size([768, 768])],
  ('index', 3)],
 [['blocks.0.ln2.b', torch.Size([768])],
  ['transformer.h.0.attn.v_proj.weight', torch.Size([768, 768])],
  ('index', 4)],
 [['blocks.0.attn.W_Q', torch.Size([12, 768, 64])],
  ['transformer.h.0.attn.q_proj.weight', torch.Size([768, 768])],
  ('index', 5)],
 [['blocks.0.attn.W_O', torch.Size([12, 64, 768])],
  ['transformer.h.0.attn.out_proj.weight', torch.Size([768, 768])],
  ('index', 6)],
 [['blocks.0.attn.b_Q', torch.Size([12, 64])],
  ['transformer.h.0.mlp.fc_in.weight', torch.Size([3072, 768])],
  ('index', 7)],
 [['blocks.0.attn.b_O', torch.Size([768])],
  ['transformer.h.0.mlp.fc_in.bias', torch.Size([3072])],
  ('index', 8)],
 [['blocks.0.attn.W_K', torch.Size([12, 768, 64])],
  ['transformer.h.0.mlp.fc_out.weight', torch.Size([768, 3072])],
  ('index', 9)],
 [['blocks.0.attn.W_V', torch.Size([12, 768, 64])],
  ['transformer.h.0.mlp.fc_out.bias', torch.S